<a href="https://colab.research.google.com/github/ameliamrnt/skills-introduction-to-github/blob/main/Poin_Absen_Tugure_(final).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

# Langkah 1: Load Data
data = pd.read_excel('attendance-report2024-10-01_2024-10-15_Wwf00nDHdiHiX0qWzLgnM2JCFvHAKfEF2738 (1).xls')

# Langkah 2: Filter Data
data_filtered = data[
    ((data['Attendance Code'] == 'H') | (data['Attendance Code'].isna())) &
    ((data['Shift'] == 'Office1') | (data['Shift'] == 'Office2'))
].copy()  # Salin data untuk menghindari SettingWithCopyWarning

# Fungsi untuk menghitung poin-poin
def calculate_points_and_columns(row):
    poin_telat = 0
    poin_pulang = 0
    poin_kosong_in = 0
    poin_kosong_out = 0
    poin_50 = 0

    try:
        # Ambil nilai kolom 'Late In', 'Early Out', dan 'Check In'
        late_in = row['Late In']
        early_out = row['Early Out']
        check_in_time = pd.to_datetime(row['Check In'], format='%H:%M', errors='coerce')
        check_in_time = check_in_time.time() if not pd.isna(check_in_time) else pd.to_datetime('00:00', format='%H:%M').time()

        # Kasus 50: Tidak check in dan check out
        if pd.isna(row['Check In']) and pd.isna(row['Check Out']):
            poin_50 = 50
        # Kasus 3: Tidak check in atau tidak check out
        elif pd.isna(row['Check In']) or pd.isna(row['Check Out']):
            if pd.isna(row['Check In']):
                poin_kosong_in = 3
            if pd.isna(row['Check Out']):
                poin_kosong_out = 3
            if late_in != '00:00':
                poin_telat = 2
            if early_out != '00:00':
                poin_pulang = 2
        else:
            # Poin 2 jika Check In lebih dari 12:00 atau Late In tidak '00:00'
            if check_in_time > pd.to_datetime('12:00', format='%H:%M').time():
                poin_telat = 2
            if late_in != '00:00':
                poin_telat = 2
            # Poin 2 jika Early Out tidak '00:00'
            if early_out != '00:00':
                poin_pulang = 2

        # Hitung total poin
        total_poin = poin_kosong_in + poin_kosong_out + poin_telat + poin_pulang + poin_50
    except Exception as e:
        print(f"Error in row {row.name}: {e}")
        total_poin = -1

    return pd.Series([poin_telat, poin_pulang, poin_kosong_in, poin_kosong_out, poin_50, total_poin])

# Langkah 3: Pastikan Kolom 'Late In' dan 'Early Out' Berisi String
data_filtered['Late In'] = data_filtered['Late In'].fillna('00:00').astype(str)
data_filtered['Early Out'] = data_filtered['Early Out'].fillna('00:00').astype(str)

# Langkah 4: Hitung Poin dan Tambahkan Kolom Baru
columns = ['poin telat', 'poin pulang', 'poin kosong in', 'poin kosong out', 'poin 50', 'total poin']
data_filtered[columns] = data_filtered.apply(calculate_points_and_columns, axis=1)

# Langkah 5: Buat Pivot Table
pivot_table = data_filtered.pivot_table(
    index=['Full Name', 'Job Position'],  # Ganti dengan nama kolom yang sesuai
    values='total poin',
    aggfunc='sum'
).reset_index()

# Langkah 6: Simpan ke File Excel dengan Dua Sheet
with pd.ExcelWriter('attendance_summary_with_pivot.xlsx', engine='openpyxl') as writer:
    # Sheet 1: Data lengkap dengan semua kolom
    data_filtered.to_excel(writer, sheet_name='Detail Points', index=False)
    # Sheet 2: Pivot table dengan Full Name, Job Position, dan total poin
    pivot_table.to_excel(writer, sheet_name='Pivot Table', index=False)

print("Data sudah disimpan sebagai 'attendance_summary_with_pivot.xlsx'")


Data sudah disimpan sebagai 'attendance_summary_with_pivot.xlsx'


In [ ]:
import pandas as pd

# Langkah 1: Load Data
data = pd.read_excel('attendance-report2024-10-01_2024-10-15_Wwf00nDHdiHiX0qWzLgnM2JCFvHAKfEF2738 (1).xls')

# Langkah 2: Filter Data
data_filtered = data[
    ((data['Attendance Code'] == 'H') | (data['Attendance Code'].isna())) &
    ((data['Shift'] == 'Office1') | (data['Shift'] == 'Office2'))
].copy()  # Salin data untuk menghindari SettingWithCopyWarning

# Fungsi untuk menghitung poin-poin
def calculate_points_and_columns(row):
    poin_telat = 0
    poin_pulang = 0
    poin_kosong_in = 0
    poin_kosong_out = 0
    poin_50 = 0

    try:
        # Ambil nilai kolom 'Late In', 'Early Out', dan 'Check In'
        late_in = row['Late In']
        early_out = row['Early Out']
        check_in_time = pd.to_datetime(row['Check In'], format='%H:%M', errors='coerce')
        check_in_time = check_in_time.time() if not pd.isna(check_in_time) else pd.to_datetime('00:00', format='%H:%M').time()

        # Kasus 3: Tidak check in atau tidak check out
        if pd.isna(row['Check In']) or pd.isna(row['Check Out']):
            if pd.isna(row['Check In']):
                poin_kosong_in = 3
            if pd.isna(row['Check Out']):
                poin_kosong_out = 3
            if late_in != '00:00':
                poin_telat = 2
            if early_out != '00:00':
                poin_pulang = 2
        else:
            # Poin 2 jika Check In lebih dari 12:00 atau Late In tidak '00:00'
            if check_in_time > pd.to_datetime('12:00', format='%H:%M').time():
                poin_telat = 2
            if late_in != '00:00':
                poin_telat = 2
            # Poin 2 jika Early Out tidak '00:00'
            if early_out != '00:00':
                poin_pulang = 2

        # Hitung total poin
        total_poin = poin_kosong_in + poin_kosong_out + poin_telat + poin_pulang + poin_50
    except Exception as e:
        print(f"Error in row {row.name}: {e}")
        total_poin = -1

    return pd.Series([poin_telat, poin_pulang, poin_kosong_in, poin_kosong_out, poin_50, total_poin])

# Langkah 3: Pastikan Kolom 'Late In' dan 'Early Out' Berisi String
data_filtered['Late In'] = data_filtered['Late In'].fillna('00:00').astype(str)
data_filtered['Early Out'] = data_filtered['Early Out'].fillna('00:00').astype(str)

# Langkah 4: Hitung Poin dan Tambahkan Kolom Baru
columns = ['poin telat', 'poin pulang', 'poin kosong in', 'poin kosong out', 'poin 50', 'total poin']
data_filtered[columns] = data_filtered.apply(calculate_points_and_columns, axis=1)

# Langkah 5: Buat Pivot Table
pivot_table = data_filtered.pivot_table(
    index=['Full Name', 'Job Position'],  # Ganti dengan nama kolom yang sesuai
    values='total poin',
    aggfunc='sum'
).reset_index()

# Langkah 6: Simpan ke File Excel dengan Dua Sheet
with pd.ExcelWriter('attendance_summary_with_pivot2.xlsx', engine='openpyxl') as writer:
    # Sheet 1: Data lengkap dengan semua kolom
    data_filtered.to_excel(writer, sheet_name='Detail Points', index=False)
    # Sheet 2: Pivot table dengan Full Name, Job Position, dan total poin
    pivot_table.to_excel(writer, sheet_name='Pivot Table', index=False)

print("Data sudah disimpan sebagai 'attendance_summary_with_pivot2.xlsx'")


Data sudah disimpan sebagai 'attendance_summary_with_pivot2.xlsx'
